In [3]:
# import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [4]:
print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')
print(f'seaborn version: {sns.__version__}')

pandas version: 1.5.2
numpy version: 1.24.1
seaborn version: 0.12.2


### Utility functions

In [ ]:
def df_unique_value(df):

    for c in df.columns:
        col_type = df[c].type

        if col_type == 'object' or col_type.name == 'category':
            print(f'{c:10}\n{df[c].unique()}')
            print('-' * 65)

def convert_col_to_category(df, cols:list):
    """ 
    convert 'cols' to 'category
    """

    for c in df[cols]:
        
        df[c] = df[c].astype('category')
        return df
    
def convert_obj_columns_tocategory(df):

    for c in df.columns:
        col_type = df[c].dtype

        if col_type == 'object' or col_type.name == 'category':
            df[c] = df[c].astype('category')
    return df

def print_category_columns(df):

    for c in df.columns:
        col_type = df[c].dtype

        if col_type.name == 'category':
            print(f'{c}: {df[c].cat.categories}')
            print(pd.Series(df[c].cat.categories))
            print(f'{c:15}: {list(enumerate(df[c].cat.categories))}')
